In [2]:
import pandas as pd


ticker = 'TSLA'

filename = '../data/' + ticker + '_1min_firstratedata.csv'
df = pd.read_csv(filename)
df['ticker'] = ticker

df["timestamp"] = pd.to_datetime(df["timestamp"])
df["date"] = df["timestamp"].dt.date
df["time"] = df["timestamp"].dt.time

In [3]:
def RSI(df, n):
    "function to calculate RSI"
    delta = df["close"].diff()
    delta = delta[1:]
    up, down = delta.copy(), delta.copy()
    up[up < 0] = 0
    down[down > 0] = 0
    df["up"] = up.round(4)
    df["down"] = down.round(4)
    AVG_Gain = df["up"].rolling(window=n).mean()
    AVG_Loss = abs(df["down"].rolling(window=n).mean())
    RS = AVG_Gain / AVG_Loss
    RSI = 100.0 - (100.0 / (1.0 + RS))
    df["RSI_14"] = RSI.round(4)
    df = df.drop(columns=["up", "down"])

    return df


df = RSI(df, 14)

df.head()

,timestamp,open,high,low,close,volume,ticker,date,time,RSI_14
0,2022-09-30 04:00:00,273.80,273.80,271.71,273.18,10243,TSLA,2022-09-30,04:00:00,NaN
1,2022-09-30 04:01:00,272.74,272.74,272.15,272.32,2963,TSLA,2022-09-30,04:01:00,NaN
2,2022-09-30 04:02:00,272.46,272.46,271.88,271.88,954,TSLA,2022-09-30,04:02:00,NaN
3,2022-09-30 04:03:00,271.88,272.00,271.75,272.00,1336,TSLA,2022-09-30,04:03:00,NaN
4,2022-09-30 04:04:00,272.21,272.27,272.02,272.27,630,TSLA,2022-09-30,04:04:00,NaN


In [4]:
# Candlestick chart of one day
import datetime
import plotly.graph_objects as go


def chart_range(df, date):
    df = df[df["date"] == date]
    fig = go.Figure(
        data=[
            go.Candlestick(
                x=df["timestamp"],
                open=df["open"],
                high=df["high"],
                low=df["low"],
                close=df["close"],
            )
        ]
    )
    fig.show()


chart_range(df, datetime.date(2023, 5, 18))

In [5]:
# chart the RSI of one day

def chart_rsi(df, date):
    df = df[df["date"] == date]
    # only chart from 9:30 to 16:00
    df = df[df["time"] >= datetime.time(9, 30, 0)]
    df = df[df["time"] <= datetime.time(16, 0, 0)]
    fig = go.Figure(data=go.Scatter(x=df["timestamp"], y=df["RSI_14"]))
    # add horizontal line at 30 and 70
    fig.add_shape(
        type="line",
        x0=df["timestamp"].min(),
        y0=30,
        x1=df["timestamp"].max(),
        y1=30,
        line=dict(color="RoyalBlue", width=1, dash="dot"),
    )
    fig.add_shape(
        type="line",
        x0=df["timestamp"].min(),
        y0=70,
        x1=df["timestamp"].max(),
        y1=70,
        line=dict(color="RoyalBlue", width=1, dash="dot"),
    )
    fig.show()


chart_rsi(df, datetime.date(2023, 5, 18))


def chart_rsi_with_trades(timestamps, rsi_values, enters, exits):
    fig = go.Figure(
        data=go.Scatter(x=timestamps, y=rsi_values, mode="lines", name="RSI")
    )
    fig.add_trace(
        go.Scatter(
            x=enters["timestamps"],
            y=enters["values"],
            mode="markers",
            marker=dict(color="green", size=8),
            name="Enter",
        )
    )
    fig.add_trace(
        go.Scatter(
            x=exits["timestamps"],
            y=exits["values"],
            mode="markers",
            marker=dict(color="red", size=8),
            name="Exit",
        )
    )
    # Add horizontal lines at RSI 30 and 70
    fig.add_shape(
        type="line",
        x0=timestamps[0],
        y0=30,
        x1=timestamps[-1],
        y1=30,
        line=dict(color="RoyalBlue", width=1, dash="dot"),
    )
    fig.add_shape(
        type="line",
        x0=timestamps[0],
        y0=70,
        x1=timestamps[-1],
        y1=70,
        line=dict(color="RoyalBlue", width=1, dash="dot"),
    )

    fig.show()

In [24]:
# Simulate a trading strategy, given a date, entry and exit RSI bounds

import datetime
import pandas as pd


def simulate_trade_RSI(date, entry, exit, df):
    # Filter data for the specified date and time range
    df_day = df[
        (df["date"] == date)
        & (df["time"] >= datetime.time(9, 30, 0))
        & (df["time"] <= datetime.time(16, 0, 0))
    ]

    # store trades in new empty dataframe
    trade_log = pd.DataFrame(
        columns=[
            "Date",
            "Direction",
            "Entry",
            "Exit",
            "TimeEnter",
            "TimeExit",
            "Gain%",
            "Gain$",
        ]
    )

    # Initialize variables
    trades = 0
    in_trade = False

    # Iterate through the dataframe
    for i in range(1, len(df_day)):
        if (
            not in_trade
            and df_day["RSI_14"].iloc[i - 1] < entry
            and df_day["RSI_14"].iloc[i] >= entry
        ):
            # Buy at entry condition

            entry_price = df_day["close"].iloc[i]
            in_trade = True
            timeEnter = df_day["time"].iloc[i - 1]

        elif (
            in_trade
            and df_day["RSI_14"].iloc[i - 1] > exit
            and df_day["RSI_14"].iloc[i] <= exit
        ):
            # Sell at exit condition
            exit_price = df_day["close"].iloc[i]

            # Calculate gain for the trade
            if entry_price != 0:
                trade_gain_percent = ((exit_price - entry_price) / entry_price) * 100
                trade_gain_percent = round(trade_gain_percent, 2)

                trade_gain_net = exit_price - entry_price
                trade_gain_net = round(trade_gain_net, 2)

                trades += 1

                # Append trade details to the trade log DataFrame
                trade_log.loc[len(trade_log)] = {
                    "Date": date,
                    "Direction": "Long",
                    "Entry": entry_price,
                    "Exit": exit_price,
                    "TimeEnter": timeEnter,
                    "TimeExit": df_day["time"].iloc[i],
                    "Gain%": trade_gain_percent,
                    "Gain$": trade_gain_net,
                }

                # Reset trade variables
                entry_price = 0
                exit_price = 0
                in_trade = False

    return trade_log


simulate_trade_RSI(datetime.date(2023, 5, 18), 30, 70, df)

,Date,Direction,Entry,Exit,TimeEnter,TimeExit,Gain%,Gain$
0,2023-05-18,Long,173.9600,175.0320,09:31:00,10:29:00,0.62,1.07
1,2023-05-18,Long,174.2681,174.3500,10:37:00,11:00:00,0.05,0.08
2,2023-05-18,Long,174.5400,175.1999,11:32:00,11:49:00,0.38,0.66
3,2023-05-18,Long,174.9950,174.7348,12:07:00,13:03:00,-0.15,-0.26
4,2023-05-18,Long,174.7600,175.2100,13:14:00,13:23:00,0.26,0.45
5,2023-05-18,Long,175.1000,175.0694,13:39:00,14:38:00,-0.02,-0.03


In [21]:

def simulate_date_range(start_date, end_date, entry, exit, df, debug=False):
    # Iterate through the date range
    sum_trades = 0
    product_gain_percent = 1
    sum_gain_net = 0

    # store trades in new empty dataframe
    day_datas = []

    for n in range((end_date - start_date).days + 1):
        date = start_date + datetime.timedelta(n)
        day_data = simulate_trade_RSI(date, entry, exit, df)
        if day_data is not None:
            #print(date, day_data)

            day_datas.append(day_data)

            sum_trades += day_data["Trades"]
            product_gain_percent *= 1 + (day_data["Gain%"])
            sum_gain_net += day_data["Gain$"]

    status = pd.DataFrame(
        {
            "Start Date": start_date,
            "End Date": end_date,
            "Total Trades": sum_trades,
            "Product Gain%": round((product_gain_percent - 1) * 100, 2),
            "Total Gain$": round(sum_gain_net, 2),
        }
    )

    # flatten day_datas into a single dataframe
    day_datas = pd.concat(day_datas, ignore_index=True)

    if debug:
        print(status)

    return day_datas


simulate_date_range(datetime.date(2023, 5, 1), datetime.date(2023, 5, 31), 30, 50, df)

,RSI Bounds,Trades,Gain%,Gain$
0,"(30, 50)",5,-1.60,-2.60
1,"(30, 50)",7,-1.55,-2.51
2,"(30, 50)",6,0.34,0.58
3,"(30, 50)",8,-0.10,-0.14
4,"(30, 50)",4,0.50,0.82
5,"(30, 50)",0,0.00,0.00
6,"(30, 50)",0,0.00,0.00
7,"(30, 50)",6,-0.55,-0.95
8,"(30, 50)",8,-0.08,-0.17
9,"(30, 50)",5,-1.10,-1.85
